### Auto block id creation

In [1]:
import mab_subjects
import numpy as np
import pandas as pd

exps = mab_subjects.unstruc.allsess + mab_subjects.struc.allsess

for exp in exps:

    task = exp.b2a.filter_by_trials(min_trials=100, clip_max=100)
    task.auto_block_window_ids()
    mask = task.block_ids == 1
    task = task._filtered(mask)

[WindowsPath('D:/Data/mab/ACdataset/Aggro/pre_lesion/Aggro_Pre.csv')]
[WindowsPath('D:/Data/mab/ACdataset/Aggro/post_lesion/Aggro_Post.csv')]
[WindowsPath('D:/Data/mab/ACdataset/Auroma/Auroma.csv')]
[WindowsPath('D:/Data/mab/ACdataset/Torres/torres.csv')]
[WindowsPath('D:/Data/mab/ACdataset/Debruyne/DeBruyne.csv')]
[WindowsPath('D:/Data/mab/ACdataset/Kompany/Kompany.csv')]
[WindowsPath('D:/Data/mab/ASdataset/grump/GrumpExp1Unstructured/GrumpExp1Unstructured.csv')]
[WindowsPath('D:/Data/mab/ACdataset/Bewilderbeast/pre_lesion/Bewilderbeast_Pre.csv')]
[WindowsPath('D:/Data/mab/ACdataset/Aguero/pre_lesion/Aguero_Pre.csv')]
[WindowsPath('D:/Data/mab/ACdataset/Aguero/post_lesion/Aguero_post.csv')]
[WindowsPath('D:/Data/mab/ACdataset/Sterling/sterling.csv')]
[WindowsPath('D:/Data/mab/ACdataset/Phil/Phil.csv')]
[WindowsPath('D:/Data/mab/ACdataset/Rodri/Rodri.csv')]
[WindowsPath('D:/Data/mab/ASdataset/gronckle/pre_lesion/Gronckle_pre.csv')]
[WindowsPath('D:/Data/mab/ASdataset/toothless/Toothles